In [ ]:
import os
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from wdf import FormFiller

os.environ["SELENIUM_MANAGER_DISABLE_TELEMETRY"] = "true"

from dotenv import load_dotenv
load_dotenv()

driver = webdriver.Chrome()
driver.get("data:,")
wait = WebDriverWait(driver, 15)

f = FormFiller(driver, wait)

In [2]:
email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

f.create_acct_or_log_in(email, password)

waiting...
title found!


In [3]:
f.fill_out_information_page()

In [4]:
with open("resume.json") as rf:
    experience_data = json.load(rf)

f.fill_out_experience_page(experience_data)

In [32]:
wait = WebDriverWait(driver, 15)
f = FormFiller(driver, wait)

In [2]:
driver = webdriver.Chrome()

JOB_PAGE_URL = """
https://nyp.wd1.myworkdayjobs.com/en-US/nypcareers/job/NYPWeill-Cornell-Medical-Center/Staff-Nurse--2SW-Neuro-SDU-ICU-Full-time-Nights_00873917/apply?clientRequestID=c5c2a454712645f2a56783277f68884a
""".strip('\n').partition('?')[0]

driver.get(JOB_PAGE_URL+'/applyManually')
wait = WebDriverWait(driver, 10)